In [1]:
import numpy as np
import seaborn as sns
import sys
#import cv2 as cv2
import glob
from PIL import Image
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import gc
import keras
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

print(tf.__version__)

2.9.1


In [2]:
#https://www.askpython.com/python-modules/python-patchify
#https://pypi.org/project/patchify/
skip = 0.5
nsize = 256
def parches(image,skip = 0.5, nsize = 256):
    patch = []
    dim_x = np.shape(image)[1]
    dim_y = np.shape(image)[0]
    ##Cantidad de imágenes que se pueden obtener vertical y horizontalmente
    npatc_x = int(np.floor(dim_x/nsize))
    npatc_y = int(np.floor(dim_y/nsize))
    
    ##Recorrido de recorte y creación del batch
    for x in range((npatc_x*2)-2):
        lim_izq = int(np.ceil((x*(nsize/2))))
        lim_der = lim_izq+nsize
        for y in range((npatc_y*2)-2):
            lim_sup = int(np.ceil((y*(nsize/2))))
            lim_inf = lim_sup+nsize
            #print(lim_sup-lim_inf,'/',lim_izq-lim_der)
            patch.append(tf.keras.preprocessing.image.img_to_array(image)[lim_sup:lim_inf,lim_izq:lim_der])
    
    return patch

In [3]:
### Función que recibe una imagen PIL de keras, y retorna la misma imagen PIL recortada
def cropping(image,crop_perc = 0.15,resize=1):
    ##Tamaño de la imágen original
    px = np.shape(image)[0]
    py = np.shape(image)[1]
    
    ##Nuevos límites verticales y horizontales
    vert_sup = int(np.ceil(px*crop_perc))
    vert_inf = int(np.ceil(px*(1-crop_perc)))
    htal_izq = int(np.ceil(py*crop_perc))
    htal_der = int(np.ceil(py*(1-crop_perc)))
    
    # Construcción de la nueva imágen
    im_array = tf.keras.preprocessing.image.img_to_array(image)
    nim_array = tf.keras.preprocessing.image.array_to_img(im_array[vert_sup:vert_inf,htal_izq:htal_der])
    
    return nim_array

In [4]:
### Rutas
classes = ['CuNi1','CuNi2','CuNi3']
route_img = []
labels = []


for clss in classes:
  cnt = 0;
  route = 'completas1/'+clss+"/*.jpg"    
  path = glob.glob(route) 
  for img in path:
    #print(img)
    route_img.append(img)    
    labels.append(clss)
    cnt = cnt +1;
  print(cnt)

images_df = pd.DataFrame()
images_df['routes'] = route_img
images_df['label'] = labels
images_df

200
200
200


,routes,label
0,completas_1/CuNi1/3-49-35X.jpg,CuNi1
1,completas_1/CuNi1/3-41-35X.jpg,CuNi1
2,completas_1/CuNi1/3-166-35X.jpg,CuNi1
3,completas_1/CuNi1/3-138-35X.jpg,CuNi1
4,completas_1/CuNi1/3-107-35X.jpg,CuNi1
...,...,...
595,completas_1/CuNi3/MUESTRA-1-700X-177.jpg,CuNi3
596,completas_1/CuNi3/MUESTRA-1-700X-41.jpg,CuNi3
597,completas_1/CuNi3/MUESTRA-1-700X-127.jpg,CuNi3
598,completas_1/CuNi3/MUESTRA-1-700X-53.jpg,CuNi3


In [5]:
for i in range(len(images_df)):
    #Capturar ruta, etiqueta y nombre de la imagen
    ruta = images_df.loc[i][0]
    etiqueta = images_df.loc[i][1]
    name = images_df.loc[i][0].split("/")[2]
    #Crear imagen
    im = tf.keras.preprocessing.image.load_img(ruta,color_mode='grayscale')
    #Cropping
    im_cropped = cropping(im)
    
    ##Ruta de salida
    output_dir = 'recortadas1/'+etiqueta+"/"+name  
    
    
    ### Se guarda la imagen en la nueva ruta
    im_cropped.save(output_dir)